In [3]:
import os
import pandas as pd
from tqdm import tqdm
import pickle


In [4]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r.pickle", "rb") as f:
    nutids_r = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [13]:
import random

step_rates = [1, 2, 3]
pre_pad_result = []
len_of_list = int(len(lines))
for i in tqdm(range(0, len_of_list//3,step_rates[0])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[0]])).split())
for i in tqdm(range(len_of_list//3, 2*len_of_list//3,step_rates[1])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[1]])).split())

random.shuffle(pre_pad_result)

100%|██████████| 328134/328134 [00:27<00:00, 11830.82it/s] 


In [14]:
scope = 10
middle = int(scope/2)
padding = int(scope/2)

In [15]:
pre_cleaned_result = [["<PAD>"]*padding + lst + ["<PAD>"]*padding for lst in tqdm(pre_pad_result)]

100%|██████████| 984400/984400 [00:39<00:00, 24998.00it/s] 


In [16]:
char = "*@;:!\"?«».,"
post_pad_result = [[word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst] for pre_cleaned_lst in tqdm(pre_cleaned_result)]

# for i in tqdm(range(len(pre_cleaned_result))):
#     pre_cleaned_lst = pre_cleaned_result[i]
#     lst = [word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst]
#     post_pad_result.append(lst)

100%|██████████| 984400/984400 [01:21<00:00, 12106.52it/s]


In [17]:
saved_post_pad_result = post_pad_result

In [18]:
post_pad_result = saved_post_pad_result[:50000]

In [19]:
bøjning = {0: "infinitiv", 1: "førnutidsformen"}
half_scope = int(scope/2)
big_lst = []
output_lst = []


for i in tqdm(range(len(post_pad_result))):
    current_lst = post_pad_result[i]
    for x in range(len(current_lst)):
        current_word = current_lst[x]
        try: stemmed_word = nutids_r_stem[current_word]
        except: continue
        current_dataset = current_lst[x-half_scope:x+half_scope+1]
        try: current_dataset[half_scope] = stemmed_word
        except: print(current_dataset); continue
        big_lst.append((" ".join(current_dataset)).lower())
        if nutids_r[stemmed_word][0] == current_word:
            output = 0
        elif nutids_r[stemmed_word][1] == current_word:
            output = 1
        else:
            print("ERROR")
        output_lst.append(output)


100%|██████████| 50000/50000 [00:01<00:00, 37801.45it/s]


In [20]:
df = pd.DataFrame()
df["comment_text"] = big_lst
df["label"] = output_lst

print(len(df))
df = df[:6000000]

def distribution(df):
    print(df["label"].value_counts())

distribution(df)
print(len(df))
header = ["comment_text", "label"]
df.to_csv("../Datasets/EuroparlNutidsr.csv", encoding="UTF-8", index=False, sep=";")

143961
0    92156
1    51805
Name: label, dtype: int64
143961


--- 
### Making testset for nutids-r module

In [5]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [6]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [12]:
df = pd.read_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", sep=";")
correct_sentences = list(df["correct"])
wrong_sentences = list(df["wrong"])
from IPython.display import clear_output

def save_df():
    df = pd.DataFrame()
    df["wrong"] = wrong_sentences
    df["correct"] = correct_sentences
    df.to_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", index=False, sep=";")

for line in tqdm(lines[122:1000]):
    line = line.strip("\n")
    words = line.split()
    for i, word in enumerate(words):
        print("hey")
        symbol = ""
        if word[-1] in ".,?!":
            symbol = word[-1]
            word = word[:-1]
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s":
            continue
        print(line)
        if nutids_r_bøjninger[stemmed][0] == word:
            bøjet_ord = nutids_r_bøjninger[stemmed][1]
        else:
            bøjet_ord = nutids_r_bøjninger[stemmed][0]
        print(f"{word} -> {bøjet_ord}")
        to_continue = int(input(f"1 for Ja, 2 for Nej."))
        clear_output()
        if to_continue == 2:
            continue
        words[i] = bøjet_ord + symbol
    new_line = " ".join(words)
    wrong_sentences.append(new_line)
    correct_sentences.append(line)
    save_df()

  2%|▏         | 16/894 [02:18<1:40:50,  6.89s/it]

hey
hey
hey
hey
hey
hey
hey
Fru formand, da jeg kan høre en smule latter fra Socialdemokraterne - jeg har fået fortalt, at brede kredse i Den Socialdemokratiske Gruppe også gerne vil have taget dette punkt af dagsordenen, fordi der ved afstemningen på Formandskonferencen ikke forelå noget votum fra arbejdsgruppen af ansvarlige kolleger i Den Socialdemokratiske Gruppe.
kan -> kunne


  2%|▏         | 16/894 [02:20<2:08:02,  8.75s/it]


ValueError: invalid literal for int() with base 10: ''

----

In [13]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [14]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [15]:
# If changed all of the dataset also needs to be changed

scope = 10
padding = int(scope/2)


In [16]:
testset = []
labels = []

In [17]:
df = pd.read_csv("../Datasets/EuroparlNutidsr_trainset.csv", encoding="UTF-8", sep=";")
testset = list(df["comment_text"])
labels = list(df["label"])

df

,comment_text,label
0,<PAD> <PAD> CCONJ NOUN ADP NOUN ADP PROPN PRON...,1
1,ADP NOUN ADP PROPN PRON VERB ADP NOUN ADP NOUN...,0
2,<PAD> <PAD> <PAD> <PAD> SCONJ PRON VERB ADP NO...,1
3,<PAD> <PAD> <PAD> SCONJ PRON VERB ADP NOUN ADP...,0
4,VERB ADP NOUN ADP NOUN ADV AUX NOUN ADP DET NOUN,1
...,...,...
292324,NOUN ADP ADJ NOUN NOUN VERB ADV NOUN ADP DET NOUN,0
292325,PRON PRON ADV ADV AUX VERB ADP DET ADJ NOUN <PAD>,1
292326,NOUN SCONJ DET NOUN AUX AUX VERB ADV ADP NOUN ...,1
292327,DET NOUN NOUN ADP NOUN VERB ADP PUNCT CCONJ PR...,0


In [18]:
import stanza

pos_model = stanza.Pipeline("da", processors='tokenize,pos', use_gpu=True, cache_directory='./cache', tokenize_pretokenized=True, n_process=8)

def get_pos_tags(words):
    doc = pos_model(" ".join(words))
    results = [word.upos for sentence in doc.sentences for i, word in enumerate(sentence.words)]
    return results

# Already done up to 143.000

for line in tqdm(lines[130000:143000]):
    line = line.strip("\n")
    true_words = line.split()
    pos = get_pos_tags(true_words)
    words = ["<PAD>"]*padding + pos + ["<PAD>"]*padding
    for i, word in enumerate(true_words):
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s":
            continue
        if nutids_r_bøjninger[stemmed][0] == word:
            labels.append(1)
        else:
            labels.append(0)
        testset.append(" ".join(words[i:i+2*padding+1]))

2023-05-04 13:49:42 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-05-04 13:49:42 INFO: Loading these models for language: da (Danish):
| Processor | Package |
-----------------------
| tokenize  | ddt     |
| pos       | ddt     |

2023-05-04 13:49:42 WARNING: GPU requested, but is not available!
2023-05-04 13:49:42 INFO: Using device: cpu
2023-05-04 13:49:42 INFO: Loading: tokenize
2023-05-04 13:49:42 INFO: Loading: pos
2023-05-04 13:49:43 INFO: Done loading processors!
100%|██████████| 13000/13000 [37:50<00:00,  5.73it/s] 


In [24]:
df = pd.DataFrame(zip(testset, labels), columns=["comment_text", "label"])

In [25]:
df

,comment_text,label
0,<PAD> <PAD> CCONJ NOUN ADP NOUN ADP PROPN PRON...,1
1,ADP NOUN ADP PROPN PRON VERB ADP NOUN ADP NOUN...,0
2,<PAD> <PAD> <PAD> <PAD> SCONJ PRON VERB ADP NO...,1
3,<PAD> <PAD> <PAD> SCONJ PRON VERB ADP NOUN ADP...,0
4,VERB ADP NOUN ADP NOUN ADV AUX NOUN ADP DET NOUN,1
...,...,...
320113,CCONJ SCONJ PRON AUX VERB ADP NOUN PUNCT <PAD>...,0
320114,<PAD> <PAD> <PAD> <PAD> ADV AUX PRON ADP NOUN ...,0
320115,VERB ADP NOUN ADJ NOUN VERB NOUN DET ADJ NOUN ADP,0
320116,PRON PUNCT PRON PRON VERB VERB ADJ NOUN ADP NO...,0


In [26]:
comment_text = df["comment_text"].values
mistakes = [l for l in comment_text if not l.isupper()]
middle = [l.split()[5] for l in comment_text]

from collections import Counter

counter = Counter(middle)
for key, value in counter.items():
    print(key, ": ", value)

NOUN :  24172
VERB :  196743
PRON :  29802
ADV :  8086
AUX :  35093
ADJ :  12606
NUM :  3310
SCONJ :  6164
ADP :  4073
X :  67
INTJ :  1
PROPN :  1


In [27]:
df.to_csv("../Datasets/EuroparlNutidsr_trainset.csv", encoding="UTF-8", index=False, sep=";")

In [28]:
verb_text_and_labels = [(c, l) for c, l in zip(comment_text, labels) if c.split()[5] == "VERB"]
verb_text = [c for c, l in verb_text_and_labels]
verb_labels = [l for c, l in verb_text_and_labels]

df = pd.DataFrame(zip(verb_text, verb_labels), columns=["comment_text", "label"])
df.to_csv("../Datasets/EuroparlNutidsr_trainset_verbs.csv", encoding="UTF-8", index=False, sep=";")
df

,comment_text,label
0,ADP NOUN ADP PROPN PRON VERB ADP NOUN ADP NOUN...,0
1,<PAD> <PAD> <PAD> SCONJ PRON VERB ADP NOUN ADP...,0
2,ADP NOUN VERB ADP PART VERB DET ADJ NOUN PUNCT...,1
3,<PAD> <PAD> <PAD> <PAD> PRON VERB ADJ NOUN ADP...,0
4,NOUN CCONJ PRON AUX ADV VERB PRON <PAD> <PAD> ...,1
...,...,...
196738,NOUN ADP NOUN ADP PART VERB DET ADJ PUNCT PRON...,1
196739,<PAD> <PAD> <PAD> <PAD> PRON VERB ADV ADP DET ...,0
196740,VERB ADP NOUN ADJ NOUN VERB NOUN DET ADJ NOUN ADP,0
196741,PRON PUNCT PRON PRON VERB VERB ADJ NOUN ADP NO...,0
